In [ ]:
from deepface import DeepFace
import cv2
from collections import Counter

# Define a function to detect faces in each frame
def detect_faces(frame):
    # Load the pre-trained face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Return the bounding boxes of the detected faces
    return faces

# Initialize the video capture
cap = cv2.VideoCapture(0)

# List to store emotion predictions
emotions = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces in the frame
    faces = detect_faces(frame)

    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract the face region
        face = frame[y:y+h, x:x+w]

        try:
            # Analyze the face using DeepFace
            results = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)

            # Extract the dominant emotion from the results
            if results and isinstance(results, list) and 'dominant_emotion' in results[0]:
                emotion = results[0]['dominant_emotion']
                emotions.append(emotion)
                
                # Draw bounding box around the face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

                # Label the face with the detected emotion
                cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

            else:
                print("Error: 'dominant_emotion' key not found in results:", results)
        except Exception as e:
            print("Error:", e)

    # Display the frame with emotion predictions and bounding boxes
    cv2.imshow('Real-time Emotion Detection', frame)

    # Check for key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture
cap.release()
cv2.destroyAllWindows()

# Generate report
if emotions:
    emotion_counts = Counter(emotions)
    total_predictions = len(emotions)
    emotion_percentages = {emotion: count / total_predictions * 100 for emotion, count in emotion_counts.items()}

    print("Emotion Predictions:")
    for emotion, percentage in emotion_percentages.items():
        print(f"{emotion}: {percentage:.2f}%")
else:
    print("No emotion predictions were made.")


---